In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector  # 用于可视化

In [2]:
# 载入数据集
mnist = input_data.read_data_sets(os.path.join(os.getcwd(), 'MNIST_data'), one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 运行次数
max_steps = 1001
# 图片数量
image_num = 3000
# 文件路径
DIR = os.getcwd()

In [4]:
# 定义会话
sess = tf.Session()

In [5]:
# 载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')

In [6]:
# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [7]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')

In [8]:
# 显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    # tf.reshape的shape的参数，第一个-1对应None，不知道多少个的向量实例
    # 28，28 表示将784的图片重新分成28*28的矩阵
    # 1 表示 维度是1，这里的图片是黑白图片
    # 如果是三原色的彩色图片，那这里的维度就是3
    
    tf.summary.image('input', image_shaped_input, 10)
    

tf.summary.image(

    name,
    tensor,
    max_outputs=3,
    collections=None,
    family=None
)

定义于：tensorflow/python/summary/summary.py

输出Summary带有图像的协议缓冲区。

构建的图像的Tensor必须是4-D形状[batch_size, height, width, channels], 其中channels可以是：

1： tensor被解析成灰度图像。

3：tensor被解析成RGB图像。

4：tensor被解析成RGBA图像

图像具有与输入张量相同的通道数。对于浮点输入，将值一次标准化为一个在[0, 255], uint8值不变。op使用两种不同的规范化算法：


	如果输入值都是正数，则重新调整它们，因此最大值为255。
	
	
	如果任何输入值为负，则移动值，使输入值0.0为127.然后重新调整它们，使得最小值为0，或者最大值为255。
	
参数：

name：生成的节点的名称。也将作为TensorBoard中的系列名称。

tensor：uint8或者float32型的4-D Tensor[batch_size, height, width, channels]，其中channels为1， 3 或者4。

max_outpus：要生成图像的最大批处理元素数。

collections：ops.GraphKeys的可选列表。要添加摘要的集合。默认为[_ops.GraphKeys.SUMMARIES]。

family：可选的; 如果提供，则用作摘要标记名称的前缀，该名称控制用于在Tensorboard上显示的选项卡名称。

返回值：

Tensor：string类型的序列化Summary协议缓冲区。

tf.summary.image('images_with_distorted_bounding_box',image_with_distorted_box)

In [9]:
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        # W = tf.Variable(tf.truncated_normal([784,10]), name='W')
        W = tf.Variable(tf.zeros([784,10]), name='W')
        variable_summaries(W)
    with tf.name_scope("biases"):
        # b = tf.Variable(tf.zeros([10])+0.1, name='b')
        b = tf.Variable(tf.zeros([10]), name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)

In [10]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)

In [11]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [12]:
sess.run(tf.global_variables_initializer())

In [13]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

In [14]:
# 产生metadata文件
if tf.gfile.Exists(DIR + '/projector/projectpr/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + '/projector/projectpr/metadata.tsv')

with open(DIR + '/projector/projector/metadata.tsv', 'w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:], 1))
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')

In [15]:
# 合并所有的summary
merged = tf.summary.merge_all()

tf.summary.merge_all()的坑
- 1. 自动管理模式    
注意：自动管理模式下，导入已保存的模型继续训练时，会抛异常，该种场景下建议使用手动模式

summary_writer = tf.summary.FileWriter('./result/',  flush_secs=60)       
summary_writer.add_graph(sess.graph)  #添加graph图     
tf.summary.scalar('loss', loss)      
tf.summary.scalar('accuracy', accuracy)   
sum_ops = tf.summary.merge_all() #自动管理    
metall = sess.run(sum_ops, feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1.})    

summary_writer.add_summary(metall, global_step=step)  # 写入文件</code>  

- 2. 手动管理模式    

summary_writer = tf.summary.FileWriter(path, flush_secs=60)     
summary_writer.add_graph(sess.graph)   
loss_scalar = tf.summary.scalar('loss', loss)    
accuracy_sacalar = tf.summary.scalar('accuracy', accuracy)    
loss_metall, accuracy_metall, = sess.run([loss_scalar, accuracy_scalar], feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1.})     

summary_writer.add_summary(loss_metall, global_step=step)    
summary_writer.add_summary(accuracy_metall, global_step = step)  

In [16]:
# embeddings可视化的主要所需文件

# 图结构
projector_writer = tf.summary.FileWriter(DIR + '/projector/projector', sess.graph)

saver = tf.train.Saver()

# 定义配置项
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + '/projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + '/projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer, config)


In [17]:
for i in range(max_steps):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary, _ = sess.run([merged, train_step] , feed_dict={x:batch_xs, y: batch_ys}, 
                          options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
    projector_writer.add_summary(summary, i)
    
    if i %100 ==0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print ("Iter " + str(i) + ", Testing Accuracy= " + str(acc))
    
saver.save(sess, DIR+'/projector/projector/a_model.ckpt', global_step=max_steps)
projector_writer.close()
sess.close()

Iter 0, Testing Accuracy= 0.3346
Iter 100, Testing Accuracy= 0.8061
Iter 200, Testing Accuracy= 0.8198
Iter 300, Testing Accuracy= 0.8299
Iter 400, Testing Accuracy= 0.8802
Iter 500, Testing Accuracy= 0.8921
Iter 600, Testing Accuracy= 0.8971
Iter 700, Testing Accuracy= 0.9019
Iter 800, Testing Accuracy= 0.9035
Iter 900, Testing Accuracy= 0.9067
Iter 1000, Testing Accuracy= 0.9062


官方embeddings可视化示例：
https://www.tensorflow.org/guide/embedding

https://www.tensorflow.org/guide/summaries_and_tensorboard